In [ ]:
!pip install langchain cohere
!pip install langchain_community
!pip install langchain_cohere

In [1]:
import cohere
from langchain_cohere import ChatCohere

In [ ]:
cohere_api_key = 'api_key'

llm = ChatCohere(
    cohere_api_key=cohere_api_key, 
    model='command-r-plus', 
    temprature=0.1
)

In [3]:
system_message = ("""
شما به عنوان یک دستیار هوشمند بانکی عمل می‌کنید.

وظیفه شما این است که نیت کاربر را تشخیص داده و اطلاعات مورد نیاز (اسلات‌ها) را استخراج کنید.

پاسخ را به صورت دقیق و در قالب زیر ارائه دهید:

نیت: [نیت کاربر]
اسلات‌ها: [اسلات‌ها به صورت دیکشنری پایتون]

اگر نیت یا اسلات‌ها را نمی‌توانید تشخیص دهید، بنویسید:

نیت: نامشخص
اسلات‌ها: '{{{{}}}}'

مثال‌ها:

کاربر: "موجودی حسابم چقدره؟"
پاسخ:
نیت: نمایش موجودی
اسلات‌ها: '{{{{}}}}'

کاربر: "می‌خوام ۵۰۰,۰۰۰ ریال به حساب 1234567890 منتقل کنم."
پاسخ:
نیت: انتقال وجه
اسلات‌ها: '{{{{'مبلغ': '۵۰۰,۰۰۰ ریال', 'شماره حساب مقصد': '1234567890'}}}}'

کاربر: "اپلیکیشن بانک همراه کار نمی‌کنه."
پاسخ:
نیت: رفع مشکل
اسلات‌ها: '{{{{'شرح مشکل': 'اپلیکیشن بانک همراه کار نمی‌کند'}}}}'

غیر از این خروجی هیچ خروجی دیگری نده
""")

In [4]:
examples = [
    {
        "question": "موجودی حسابم چقدره؟",
        "intent": "نمایش موجودی",
        "slots": "{}"
    },
    {
        "question": "می‌خوام ۵۰۰,۰۰۰ ریال به حساب 1234567890 منتقل کنم.",
        "intent": "انتقال وجه",
        "slots": "{'مبلغ': '۵۰۰,۰۰۰ ریال', 'شماره حساب مقصد': '1234567890'}"
    },
    {
        "question": "آخرین تراکنش‌هامو می‌خوام ببینم.",
        "intent": "نمایش تاریخچه تراکنش‌ها",
        "slots": "{}"
    },
    {
        "question": "اپلیکیشن بانک همراه کار نمی‌کنه.",
        "intent": "رفع مشکل",
        "slots": "{'شرح مشکل': 'اپلیکیشن بانک همراه کار نمی‌کند'}"
    },
    {
        "question": "چطور می‌تونم پول انتقال بدم؟",
        "intent": "انتقال وجه",
        "slots": "{}"
    },
    {
        "question": "مشکل ورود به حساب دارم.",
        "intent": "رفع مشکل",
        "slots": "{'شرح مشکل': 'مشکل در ورود به حساب'}"
    },
    {
        "question": "میرم داخل پروفایلم عکسمو نشون نمیده",
        "intent": "رفع مشکل",
        "slots": "{'شرح مشکل': 'مشکل در دیدن عکس پروفایل'}"
    },
]


In [5]:
from langchain.prompts import ChatPromptTemplate
from langchain.prompts import FewShotChatMessagePromptTemplate

example_prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "{question}"),
        ("ai", "نیت: {intent}\nاسلات‌ها: {slots}")
    ]
)

In [6]:
few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
)

In [78]:
prompt_template = ChatPromptTemplate.from_messages([
    ("system", system_message),
    few_shot_prompt,
    ("human", "{question}")
])

In [116]:
from langchain.tools import tool

@tool
def get_balance_tool(account_id: str) -> dict:
    """مقدار موجودی کاربر را برمی گرداند"""
    accounts = {
        "A1001": {"customer_id": "C1001", "balance": 10000},
        "A1002": {"customer_id": "C1002", "balance": 15000},
    }
    if account_id not in accounts:
        return {"error": "Account not found"}
    return {"account_id": account_id, "balance": accounts[account_id]["balance"]}

@tool
def transfer_funds_tool(from_account: str, to_account: str, amount: float) -> dict:
    """پول را از حساب شخص منتقل میکند به حساب شخص دیگر"""
    accounts = {
        "A1001": {"customer_id": "C1001", "balance": 10000000},
        "A1002": {"customer_id": "C1002", "balance": 15000000},
    }
    if from_account not in accounts or to_account not in accounts:
        return {"error": "Invalid account details"}
    if accounts[from_account]["balance"] < amount:
        return {"error": "Insufficient funds"}
    
    accounts[from_account]["balance"] -= amount
    accounts[to_account]["balance"] += amount
    
    return {
        "status": "success",
        "from_account_balance": accounts[from_account]["balance"],
        "to_account_balance": accounts[to_account]["balance"],
    }

transactions = [
    {"transaction_id": "T1001", "from_account": "A1001", "to_account": "A1002", "amount": 2000, "date": "2024-11-01"},
    {"transaction_id": "T1002", "from_account": "A1002", "to_account": "A1001", "amount": 1500, "date": "2024-11-02"},
    {"transaction_id": "T1003", "from_account": "A1001", "to_account": "A1003", "amount": 3000, "date": "2024-11-03"},
]

@tool
def transaction_history_tool(account_id: str) -> dict:
    """تاریخچه تراکنش‌های مربوط به یک کاربر خاص را برمی‌گرداند"""
    filtered_transactions = [
        transaction for transaction in transactions
        if transaction["from_account"] == account_id or transaction["to_account"] == account_id
    ]
    
    if not filtered_transactions:
        return {"error": "No transactions found for the given account"}
    
    return {"account_id": account_id, "transactions": filtered_transactions}


In [118]:
tools = [get_balance_tool, transfer_funds_tool, transaction_history_tool]

In [119]:
from langchain.chains import LLMChain

llm_with_tools = llm.bind_tools(tools)

chain = prompt_template | llm_with_tools

In [120]:
def print_output(response):
    if response[0]['name'] == "get_balance_tool":
        return get_balance_tool(response[0]['args']['account_id'])
    elif response[0]['name'] == "transfer_funds_tool":
        return transfer_funds_tool({
            "from_account": response[0]['args']['from_account'],
            "to_account": response[0]['args']['to_account'],
            "amount": response[0]['args']['amount']
        })
    elif response[0]['name'] == "transaction_history_tool":
        return transaction_history_tool(response[0]['args']['account_id'])

# مثال اول

In [121]:
prompt = "موجودی A1001 بگو"
response = chain.invoke(prompt).tool_calls

In [122]:
response

[{'name': 'get_balance_tool',
  'args': {'account_id': 'A1001'},
  'id': '7afeb5a04ba342d7811f56bd2ddd64b6',
  'type': 'tool_call'}]

In [123]:
print_output(response)

{'account_id': 'A1001', 'balance': 10000}

# مثال دوم

In [124]:
prompt = "1000 از حساب A1001 بزن ب حساب A1002"
response = chain.invoke(prompt).tool_calls

In [125]:
response

[{'name': 'transfer_funds_tool',
  'args': {'amount': 1000, 'from_account': 'A1001', 'to_account': 'A1002'},
  'id': '6fe471d0570b43bbac571858cf90440c',
  'type': 'tool_call'}]

In [126]:
print_output(response)

{'status': 'success',
 'from_account_balance': 9999000.0,
 'to_account_balance': 15001000.0}

# مثال سوم

In [127]:
prompt = "تاریخچه تراکنش های کاربر A1001 بهم بده"
response = chain.invoke(prompt).tool_calls

In [128]:
response

[{'name': 'transaction_history_tool',
  'args': {'account_id': 'A1001'},
  'id': 'cd3d915c94664593b098fc2e8411e862',
  'type': 'tool_call'}]

In [129]:
print_output(response)

{'account_id': 'A1001',
 'transactions': [{'transaction_id': 'T1001',
   'from_account': 'A1001',
   'to_account': 'A1002',
   'amount': 2000,
   'date': '2024-11-01'},
  {'transaction_id': 'T1002',
   'from_account': 'A1002',
   'to_account': 'A1001',
   'amount': 1500,
   'date': '2024-11-02'},
  {'transaction_id': 'T1003',
   'from_account': 'A1001',
   'to_account': 'A1003',
   'amount': 3000,
   'date': '2024-11-03'}]}

# مثال چهارم

In [130]:
prompt = "وقتی کلیک میکنم هچی واسم نشون نمیده"
response = chain.invoke({"question":prompt})

In [132]:
print(response.content)

نیت: رفع مشکل
اسلات‌ها: {'شرح مشکل': 'وقتی روی چیزی کلیک می‌کنم، چیزی نشان داده نمی‌شود'}


# مثال پنجم

In [139]:
prompt = "پروفایلم سیاهه نشون نمیده انگار"
response = chain.invoke({"question":prompt})

In [140]:
response

AIMessage(content="نیت: رفع مشکل\nاسلات\u200cها: {'شرح مشکل': 'پروفایل سیاه است'}", additional_kwargs={'documents': None, 'citations': None, 'search_results': None, 'search_queries': None, 'is_search_required': None, 'generation_id': '0e696e26-3552-433a-99ba-7321ffc60c56', 'token_count': {'input_tokens': 1754.0, 'output_tokens': 65.0}}, response_metadata={'documents': None, 'citations': None, 'search_results': None, 'search_queries': None, 'is_search_required': None, 'generation_id': '0e696e26-3552-433a-99ba-7321ffc60c56', 'token_count': {'input_tokens': 1754.0, 'output_tokens': 65.0}}, id='run-3b840a0e-83c7-4ac7-81a3-5e4df93762ff-0', usage_metadata={'input_tokens': 1754, 'output_tokens': 65, 'total_tokens': 1819})

In [141]:
print(response.content)

نیت: رفع مشکل
اسلات‌ها: {'شرح مشکل': 'پروفایل سیاه است'}
